In [18]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [30]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Load MNIST dataset
mnist = fetch_openml(name='mnist_784', version=1)
X, Y = mnist.data, mnist.target.astype(int)

# Normalize pixel values
X = X / 255.

# Shuffle and split the dataset into training and development sets
X, Y = shuffle(X, Y, random_state=42)
X_train, X_dev, Y_train, Y_dev = train_test_split(X, Y, test_size=0.1, random_state=42)

m_train, n = X_train.shape


C:\Users\juvva\anaconda3\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Load MNIST dataset
mnist = fetch_openml(name='mnist_784', version=1)
X, Y = mnist.data, mnist.target.astype(int)

# Normalize pixel values
X = X / 255.

# Shuffle and split the dataset into training and development sets
X, Y = shuffle(X, Y, random_state=42)
X_train, X_dev, Y_train, Y_dev = train_test_split(X, Y, test_size=0.1, random_state=42)

m_train, n = X_train.shape

In [39]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return (Z > 0).astype(float)

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

def adam_optimizer(W1, b1, W2, b2, dW1, db1, dW2, db2, m_W1, v_W1, m_b1, v_b1, m_W2, v_W2, m_b2, v_b2, t, alpha=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
    m_W1 = beta1 * m_W1 + (1 - beta1) * dW1
    v_W1 = beta2 * v_W1 + (1 - beta2) * (dW1 ** 2)
    m_b1 = beta1 * m_b1 + (1 - beta1) * db1
    v_b1 = beta2 * v_b1 + (1 - beta2) * (db1 ** 2)
    m_W2 = beta1 * m_W2 + (1 - beta1) * dW2
    v_W2 = beta2 * v_W2 + (1 - beta2) * (dW2 ** 2)
    m_b2 = beta1 * m_b2 + (1 - beta1) * db2
    v_b2 = beta2 * v_b2 + (1 - beta2) * (db2 ** 2)

    m_W1_hat = m_W1 / (1 - beta1 ** t)
    v_W1_hat = v_W1 / (1 - beta2 ** t)
    m_b1_hat = m_b1 / (1 - beta1 ** t)
    v_b1_hat = v_b1 / (1 - beta2 ** t)
    m_W2_hat = m_W2 / (1 - beta1 ** t)
    v_W2_hat = v_W2 / (1 - beta2 ** t)
    m_b2_hat = m_b2 / (1 - beta1 ** t)
    v_b2_hat = v_b2 / (1 - beta2 ** t)

    W1 = W1 - alpha * m_W1_hat / (np.sqrt(v_W1_hat) + epsilon)
    b1 = b1 - alpha * m_b1_hat / (np.sqrt(v_b1_hat) + epsilon)
    W2 = W2 - alpha * m_W2_hat / (np.sqrt(v_W2_hat) + epsilon)
    b2 = b2 - alpha * m_b2_hat / (np.sqrt(v_b2_hat) + epsilon)

    return W1, b1, W2, b2, m_W1, v_W1, m_b1, v_b1, m_W2, v_W2, m_b2, v_b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size



# Gradient descent optimization
def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    m_W1, v_W1, m_b1, v_b1 = np.zeros_like(W1), np.zeros_like(W1), np.zeros_like(b1), np.zeros_like(b1)
    m_W2, v_W2, m_b2, v_b2 = np.zeros_like(W2), np.zeros_like(W2), np.zeros_like(b2), np.zeros_like(b2)

    for i in range(1, iterations + 1):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2, m_W1, v_W1, m_b1, v_b1, m_W2, v_W2, m_b2, v_b2 = adam_optimizer(
            W1, b1, W2, b2, dW1, db1, dW2, db2,
            m_W1, v_W1, m_b1, v_b1, m_W2, v_W2, m_b2, v_b2,
            i, alpha=alpha
        )
    return W1, b1, W2, b2

# Run the gradient descent optimization
W1, b1, W2, b2 = gradient_descent(X_train.T, Y_train, alpha=0.001, iterations=500)

# Evaluate the accuracy on the development set
dev_predictions = get_predictions(forward_prop(W1, b1, W2, b2, X_dev.T)[-1])
dev_accuracy = get_accuracy(dev_predictions, Y_dev)
print(f"Accuracy on Development Set: {dev_accuracy}")


Accuracy on Development Set: 0.8867142857142857
